# Crop types (Switzerland) dataset extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is used to retrieve and concatenate the crop-types dataset into a table for publication alongisde the used data.

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas=2.1.3
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* Crops_Timeseries_1980_2019.xlsx


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* 
## Observations
* 

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import tqdm as tqdm
import os
import warnings
import geopandas as gpd

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../.."
# Suppress all warnings
warnings.filterwarnings("ignore")

path_data = r"C:\Users\nascimth\Documents\data\CAMELS_CH_Chem\data"

* #### The users should NOT change anything in the code below here. 

In [ ]:
# Non-editable variables:
PATH_OUTPUT = r"results\Dataset\catchment_aggregated_data\agricultural_data"

# Set the directory:
os.chdir(PATH)

# Import data
* Full table

In [ ]:
Crops_Timeseries_1980_2019 = gpd.read_file(path_data+'\\agriculture\Crops_Timeseries_1980_2019.csv')
Crops_Timeseries_1980_2019["bafu_id"] = Crops_Timeseries_1980_2019["gauge_id"]
Crops_Timeseries_1980_2019

In [ ]:
# Network CAMELS_CH_Chem

network_camels_ch_chem = pd.read_csv(r"results\Dataset\gauges_metadata\camels_ch_chem_gauges_metadata.csv", encoding='utf-8')
#network_camels_ch_chem.set_index("basin_id", inplace=True)
network_camels_ch_chem

In [ ]:
Crops_Timeseries_1980_2019["nawaf_id"] = Crops_Timeseries_1980_2019["naduf_id"]
Crops_Timeseries_1980_2019["nawat_id"] = Crops_Timeseries_1980_2019["nawa_id"]
Crops_Timeseries_1980_2019["sensor_id"] = Crops_Timeseries_1980_2019["bafu_id"]

In [ ]:
Crops_Timeseries_1980_2019

In [ ]:
Crops_Timeseries_1980_2019['bafu_id'] = Crops_Timeseries_1980_2019['bafu_id'].replace("<Null>", 0)
Crops_Timeseries_1980_2019['nawat_id'] = Crops_Timeseries_1980_2019['nawat_id'].replace("<Null>", 0)
Crops_Timeseries_1980_2019['nawaf_id'] = Crops_Timeseries_1980_2019['nawaf_id'].replace("<Null>", 0)
Crops_Timeseries_1980_2019['sensor_id'] = Crops_Timeseries_1980_2019['sensor_id'].replace("<Null>", 0)
Crops_Timeseries_1980_2019['gauge_id'] = Crops_Timeseries_1980_2019['gauge_id'].replace("<Null>", 0)


Crops_Timeseries_1980_2019

In [ ]:
network_camels_ch_chem["basin_id"] = network_camels_ch_chem["gauge_id"]

In [ ]:
Crops_Timeseries_1980_2019

In [ ]:
Crops_Timeseries_1980_2019

In [ ]:
network_camels_ch_chem[['sensor_id', 'basin_id']]

In [ ]:
network_camels_ch_chem["sensor_id"] = network_camels_ch_chem["sensor_id"].astype(float)
network_camels_ch_chem["basin_id"] = network_camels_ch_chem["basin_id"].astype(float)


network_camels_ch_chem[['sensor_id', 'basin_id']].dtypes

In [ ]:
Crops_Timeseries_1980_2019["sensor_id"] = Crops_Timeseries_1980_2019["sensor_id"].astype(float)
Crops_Timeseries_1980_2019["nawaf_id"] = Crops_Timeseries_1980_2019["nawaf_id"].astype(float)
Crops_Timeseries_1980_2019["nawat_id"] = Crops_Timeseries_1980_2019["nawat_id"].astype(float)
Crops_Timeseries_1980_2019["bafu_id"] = Crops_Timeseries_1980_2019["bafu_id"].astype(float)
Crops_Timeseries_1980_2019["gauge_id"] = Crops_Timeseries_1980_2019["gauge_id"].astype(float)

In [ ]:
# Merge the DataFrames for achieving the bafu_id ias the last column
Crops_Timeseries_1980_2019 = pd.merge(Crops_Timeseries_1980_2019, network_camels_ch_chem[['sensor_id', 'basin_id']], on='sensor_id', how='left')
Crops_Timeseries_1980_2019 = pd.merge(Crops_Timeseries_1980_2019, network_camels_ch_chem[['nawaf_id', 'basin_id']], on='nawaf_id', how='left')
Crops_Timeseries_1980_2019 = pd.merge(Crops_Timeseries_1980_2019, network_camels_ch_chem[['nawat_id', 'basin_id']], on='nawat_id', how='left')

Crops_Timeseries_1980_2019

In [ ]:
# Replace all 0s with NaN:
Crops_Timeseries_1980_2019.loc[:, ['gauge_id', "bafu_id", "nawaf_id", "nawat_id", "sensor_id"]] = Crops_Timeseries_1980_2019.loc[:, ['gauge_id', "bafu_id", "nawaf_id", "nawat_id", "sensor_id"]].replace(0, np.nan)

In [ ]:
# Create the new 'basin_id' column based on the priority order
Crops_Timeseries_1980_2019['basin_id_new'] = np.nan
Crops_Timeseries_1980_2019['basin_id_new'] = np.where(
    Crops_Timeseries_1980_2019['gauge_id'].notna(), Crops_Timeseries_1980_2019['basin_id_x'],
    np.where(
        Crops_Timeseries_1980_2019['nawaf_id'].notna(), Crops_Timeseries_1980_2019['basin_id_y'],
        Crops_Timeseries_1980_2019['basin_id']
    )
)

# Display the updated DataFrame
Crops_Timeseries_1980_2019

In [ ]:
# Here we can check the data
Crops_Timeseries_1980_2019[Crops_Timeseries_1980_2019.basin_id_new.isna()]

In [ ]:
# Here we solve it manuallly
Crops_Timeseries_1980_2019.loc[129, ["basin_id_new"]] = 2622.0
Crops_Timeseries_1980_2019.loc[12, ["basin_id_new"]] = 2403.0

Crops_Timeseries_1980_2019[Crops_Timeseries_1980_2019.basin_id_new.isna()]

In [ ]:
# Drop duplicates based on 'gauge_id'
Crops_Timeseries_1980_2019_unique = Crops_Timeseries_1980_2019.drop_duplicates(subset="basin_id_new")
Crops_Timeseries_1980_2019_unique.set_index("basin_id_new", inplace=True)
Crops_Timeseries_1980_2019_unique

In [ ]:
Crops_Timeseries_1980_2019_unique.drop(2622.0, axis=0, inplace=True)
Crops_Timeseries_1980_2019_unique.drop(2403.0, axis=0, inplace=True)

Crops_Timeseries_1980_2019_unique

In [ ]:
Crops_Timeseries_1980_2019_unique

In [ ]:
crops_df = pd.DataFrame(index=network_camels_ch_chem.basin_id.astype(float))

crops_df[Crops_Timeseries_1980_2019_unique.columns[13:283]] = Crops_Timeseries_1980_2019_unique[Crops_Timeseries_1980_2019_unique.columns[13:283]]

In [ ]:
# Reset the index to have basin_id as a regular column
df = crops_df.reset_index()

# Melt dataframe to long format for easier manipulation
df_long = df.melt(id_vars='basin_id', var_name='variable_year', value_name='value')

# Split variable_year into 'variable' and 'year'
df_long[['variable', 'year']] = df_long['variable_year'].str.rsplit("_", n=1, expand=True)
df_long.value = df_long.value.astype(float)

# Pivot the dataframe to have a neat time series structure
df_pivot = df_long.pivot_table(
    index=['basin_id', 'year'], columns='variable', values='value'
).reset_index()

# Loop over each unique basin_id and save as separate CSV files
for basin_id, basin_df in tqdm.tqdm(df_long.groupby('basin_id')):
    # Pivot to wide format (variables as columns, years as rows)
    df_pivot = basin_df.pivot(index='year', columns='variable', values='value')
    df_pivot.columns = ['total_arable', 'cereal', 'grapevine', 'maize', 'orchard', 'potato', 'pulse',
       'rapeseed', 'sugarbeet', 'vegetable']
    
    df_pivot = df_pivot[['cereal', 'maize', 'sugarbeet', 'potato', 'rapeseed', 'pulse',
        'vegetable', 'total_arable', 'grapevine', 'orchard']]
    
    df_pivot.index = df_pivot.index.astype(int)

    # Generate a full range of years from 1980 to 2019
    full_range = pd.DataFrame(index=range(1980, 2020))

    # Reindex the dataframe to include all years
    df_pivot_interpolated = df_pivot.reindex(full_range.index)

    # Interpolate missing values
    df_pivot_interpolated = df_pivot_interpolated.interpolate(method='linear')


    df_pivot_interpolated = df_pivot_interpolated.round(4)
    df_pivot_interpolated.index.name = "date"

    # Repeat the last row for 2020
    df_pivot_interpolated.loc[2020] = df_pivot_interpolated.loc[2019]

    df_pivot_interpolated.to_csv(PATH_OUTPUT + "/camels_ch_chem_swisscrops_"+str(int(basin_id))+".csv", encoding='latin')

# End